In [1]:
import sys
sys.path.append('..')

In [2]:
import numpy as np
import bwsample as bws

# Toy Data

In [3]:
data = (
    ([1, 0, 0, 2], ['A', 'B', 'C', 'D']),
    ([1, 0, 0, 2], ['A', 'B', 'C', 'D']), 
    ([2, 0, 0, 1], ['A', 'B', 'C', 'D']), 
    ([0, 1, 2, 0], ['A', 'B', 'C', 'D']),
    ([0, 1, 0, 2], ['A', 'B', 'C', 'D']),
)

"""
data = (
    ([1, 0, 0, 2], ['A', 'B', 'C', 'D']),
    ([1, 0, 0, 2], ['A', 'B', 'C', 'D']), 
    ([2, 0, 0, 1], ['A', 'B', 'C', 'D']), 
    ([1, 2, 0, 0], ['D', 'E', 'F', 'A']),
    ([0, 2, 1, 0], ['D', 'E', 'F', 'A']),
    ([0, 0, 1, 2], ['D', 'E', 'F', 'A'])
)
"""

# Extract pair frequencies
dok, _, _, _ = bws.extract_pairs_batch2(data)

In [4]:
# convert to sparse matrix
cnt, _ = bws.to_scipy(dok)
print(cnt.todense().astype(int))

[[0 2 3 3]
 [3 0 2 4]
 [1 0 0 3]
 [1 1 2 0]]


# Simple Ratios
Ranks and scores based on *simple ratios* are computed as follows:

1. Compute all ratios $\mu_{ij} = \frac{N_{ij}}{N_{ij} + N_{ji}} \; \forall i,j$
2. Compute the row sums $s_i = \sum_j \mu_{ij}$
3. Calibrate the values $s_i$ by Platt-Scaling as scores

Further notes: The *simple ratio* approach ignores the sample sizes $N_{ij} + N_{ji}$
across different pairs $(i,j)$.


In [5]:
ranked, ordids, scores, ratios = bws.rank(dok, method='ratio', avg='exist', calibration='platt')

print(ratios.todense().round(3), "\n")
print(f"positions: {ranked}") 
print(f"ordered IDs: {ordids}") 
print(f"scores: {scores}") 

[[0.   0.4  0.75 0.75]
 [0.6  0.   1.   0.8 ]
 [0.25 0.   0.   0.6 ]
 [0.25 0.2  0.4  0.  ]] 

positions: [1, 0, 2, 3]
ordered IDs: ['B', 'A', 'C', 'D']
scores: [0.5309053173967123, 0.5137516046973882, 0.47767157940158145, 0.47767157940158145]


# p-values based on Chi-Squared test
The question which opposing frequency $N_{ij}$ or $N_{ji}$ is larger,
can be treated as hypothesis test:

$$
\mu = \frac{N_{ij}}{N_{ij} + N_{ji}}
\quad , \quad
H_0: \mu = 0.5
\quad , \quad
H_a: \mu > 0.5
$$

The Pearson's $\chi^2$-test is implemented as alternative to the binomal test with its discrete distribution.

1. Compute *p-value based* metric $x_{ij}$. Using $1-p$ allows to store a sparse matrix as we expect many pairs $(i,j)$ having no user evaluation at all.
$$
x_{ij} = 
\left \{
\begin{aligned}
& 1-p_{ij}, & \text{if} \, N_{ij} > N_{ji} \\
& 0, & \text{otherwise}
\end{aligned} 
\right.
\quad
\forall i,j
$$
2. Sum each row $r_i = \sum_j x_{ij}$ and divide it by the actual number of row elements $n_i$
3. Calibrate the values $r_i/n_i$ by Platt-Scaling as scores

Further notes: We basically want to run [binomal test](https://en.wikipedia.org/wiki/Binomial_test) for an coin tossing type of experiment. 
When $N_{ij} > N_{ji}$, the lower the p-value of the Pearson Chi-Squared test, the more significant is the rejection of H0.
In other words, the higher $x_{ij}$, the more signification is the $N_{ij} > N_{ji}$

In [6]:
ranked, ordids, scores, minuspvalue = bws.rank(dok, method='pvalue', avg='exist', calibration='platt')

print(minuspvalue.todense().round(3), "\n")
print(f"positions: {ranked}") 
print(f"ordered IDs: {ordids}") 
print(f"scores: {scores}") 

[[0.    0.    0.683 0.683]
 [0.345 0.    0.843 0.82 ]
 [0.    0.    0.    0.345]
 [0.    0.    0.    0.   ]] 

positions: [1, 0, 2, 3]
ordered IDs: ['B', 'A', 'C', 'D']
scores: [0.5422497422485285, 0.5170970398066642, 0.47705924601529226, 0.46354837474491195]


### Notes
Why should we care about the size of $N$?
A small gap between $N_{ij}=1$ vs $N_{ji}=2$ might be interpretated big ratio gap $1/3$ vs $2/3$.
However, when collecting more data it might turn out $N_{ij}=1000$ vs $N_{ji}=1001$

In [7]:
import scipy.stats

for f1 in (1, 10, 100, 1000):
    f2 = f1 + 1
    fe = (f1 + f2) / 2
    _, pval = scipy.stats.chisquare(f_obs=[f1, f2], f_exp=[fe, fe], ddof=0)
    print(f"p-value={pval:5.3f} | Nij={f1} vs Nji={f2}")

p-value=0.564 | Nij=1 vs Nji=2
p-value=0.827 | Nij=10 vs Nji=11
p-value=0.944 | Nij=100 vs Nji=101
p-value=0.982 | Nij=1000 vs Nji=1001


# Eigenvector as scores
The idea is to solve pairwise comparison matrix as Eigenvalue-problem whereas the eigenvector can be interpreted as the items' scores [(Saaty, 2003)](http://dx.doi.org/10.1016/S0377-2217(02)00227-8) .

1. Create a reciprocal pairwise comparison matrix $A=(a_{ij})$ with 
$$
a_{ij} = 
\left \{
\begin{aligned}
& N_{ij} / N_{ji}, & \text{if} \, N_{ji} > 0 \\
& 0, & \text{otherwise}
\end{aligned} 
\right.
\quad
\forall i,j
$$
2. Solve the Eigenvalue-problem $A x = m x$ with $m$ the eigenvalue and $x=[x_1, x_2, ...,x_N]$ the eigenvector,
3. Calibrate the eigenvector $x$ with Platt-Scaling as scores

In [8]:
ranked, ordids, scores, (eigval, eigvec) = bws.rank(dok, method='eigen', calibration=None)

print(f"eigenvector: {np.abs(np.real(eigvec.reshape(1,-1)))}")

print(f"positions: {ranked}") 
print(f"ordered IDs: {ordids}") 
print(f"scores: {scores}") 

eigenvector: [[0.65510149 0.69815317 0.20364194 0.2048271 ]]
positions: [1, 0, 3, 2]
ordered IDs: ['B', 'A', 'D', 'C']
scores: [0.6981531725668787, 0.6551014935159343, 0.20482709915914155, 0.2036419413530658]


# Estimate and simulate a transition matrix
Approach:

1. Compute a transition probability matrix $\Pr(k|j)$ of items $e$ being evaluated $e_k > e_j$
2. Simulate the transition matrix
    - The initial items are equally distributed with item probability $\pi_j = 1/N \; \forall j$.
    - Predict the probability of the items $\pi_k = \pi_j \cdot \Pr(k|j)$
3. Calibrate the item probabilities $\pi_k$ to scores. Run Platt-Scaling against binary labels $y=1_{\pi_k>1/N}$

Settings:

- It is possible to skip the calibration step (3) by `calibration=None`.
- Isotonic Regression `calibration='isotonic'` should be used for large numbers items $N>1000$.
- `n_rounds` is number of simulation steps to compute $\pi_k$


In [9]:
ranked, ordids, scores, (x, transmat) = bws.rank(dok, method='transition', n_rounds=3, calibration='platt')

print(transmat.todense().round(3))
print(f"predicted probabilities: {x}", "\n") 

print(f"positions: {ranked}") 
print(f"ordered IDs: {ordids}") 
print(f"scores: {scores}") 

[[0.488 0.278 0.098 0.136]
 [0.299 0.484 0.05  0.167]
 [0.237 0.197 0.409 0.156]
 [0.207 0.224 0.134 0.435]]
predicted probabilities: [0.33187441 0.31953262 0.13568109 0.21291187] 

positions: [0, 1, 3, 2]
ordered IDs: ['A', 'B', 'D', 'C']
scores: [0.5030788667074934, 0.5026147386196117, 0.49860502987470134, 0.4957006782793525]
